In [78]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
from bokeh.io import output_notebook
output_notebook()
backtesting.set_bokeh_output(notebook=True)

Loading BokehJS ...

In [79]:
tsla_data = pd.read_csv("data/TSLA.csv")
tsla_data["Date"] = pd.to_datetime(tsla_data["Date"])
tsla_data = tsla_data[(tsla_data["Date"]>= '01/01/2022') & (tsla_data["Date"]>'01/01/2023')]
tsla_data.set_index("Date", inplace = True)

In [80]:
class MA_crossover_bt(Strategy):
    short_period = 10
    long_period = 50

    def init(self):
        # Inicializando as médias móveis: curta e longa
        self.short_ma = self.I(self.moving_average, self.data.Close, self.short_period)
        print(self.data.Close)
        self.long_ma = self.I(self.moving_average, self.data.Close, self.long_period)

    def moving_average(self, data, size):
        """Função para calcular a média móvel exponencial dos dados"""
        return pd.Series(data).ewm(span=size, adjust=False).mean()

    def next(self):
        if crossover(self.short_ma, self.long_ma):
            self.position.close()
            self.buy()
        elif crossover(self.long_ma, self.short_ma):
            self.position.close()
            self.sell()

In [81]:
# Parâmetros da estratégia
short_period = 10
long_period = 50

# Executando o backtest
bt = Backtest(tsla_data, MA_crossover_bt, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

[108.099998 113.639999 110.339996 113.059998 119.769997 118.849998
 123.220001 123.559998 122.400002 131.490005 128.779999 127.169998
 133.419998 143.75     143.889999 144.429993 160.270004 177.899994
 166.660004 173.220001 181.410004 188.270004 189.979996 194.759995
 196.809998 201.289993 207.320007 196.889999 194.639999 209.25
 214.240005 202.039993 208.309998 197.369995 200.860001 202.070007
 196.880005 207.630005 205.710007 202.770004 190.899994 197.789993
 193.809998 187.710007 182.       172.919998 173.440002 174.479996
 183.259995 180.449997 184.130005 180.130005 183.25     197.580002
 191.149994 192.220001 190.410004 191.809998 189.190002 193.880005
 195.279999 207.460007 194.770004 192.580002 185.520004 185.059998
 184.509995 186.789993 180.539993 185.899994 185.       187.039993
 184.309998 180.589996 162.990005 165.080002 162.550003 160.669998
 153.75     160.190002 164.309998 161.830002 160.309998 160.610001
 161.199997 170.059998 171.789993 169.149994 168.539993 172.080002

In [82]:
stats.iloc[0]

Timestamp('2023-01-03 00:00:00')

In [83]:
bt.plot()


C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p4846', ...)

In [84]:
class MACD(Strategy):
    """ ADD stop loss"""
    short_period = 12
    long_period = 26   
    signal_period = 9 
    

    def init(self):
        # Calculate short and long moving averages
        self.short_ma = self.I(self.moving_average, self.data.Close, self.short_period)
        self.long_ma = self.I(self.moving_average, self.data.Close, self.long_period)
        
        # MACD line: difference between short and long moving averages
        self.macd_line = self.short_ma - self.long_ma
        
        # Signal line: EMA of the MACD line
        self.signal_line = self.I(self.moving_average, self.macd_line, self.signal_period)

    def moving_average(self, data, size):
        """Exponential moving average"""
        return pd.Series(data).ewm(span=size, adjust=False).mean()

    def next(self):
        # Buy when MACD line crosses above the signal line
        if crossover(self.macd_line, self.signal_line):
            if self.position.is_short:
                self.position.close()  # Close any short positions before buying
            self.buy()

        # Sell when MACD line crosses below the signal line
        elif crossover(self.signal_line, self.macd_line):
            if self.position.is_long:
                self.position.close()  # Close any long positions before selling
            self.sell()

In [85]:
# Executando o backtest
bt = Backtest(tsla_data, MACD, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

Start                     2023-01-03 00:00:00
End                       2024-08-21 00:00:00
Duration                    596 days 00:00:00
Exposure Time [%]                   91.727494
Equity Final [$]                 24351.683074
Equity Peak [$]                  27672.734767
Return [%]                         143.516831
Buy & Hold Return [%]              106.540248
Return (Ann.) [%]                   72.582149
Volatility (Ann.) [%]               87.264191
Sharpe Ratio                         0.831752
Sortino Ratio                         2.44232
Calmar Ratio                         2.611953
Max. Drawdown [%]                   -27.78846
Avg. Drawdown [%]                   -6.623716
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                   26
Win Rate [%]                        57.692308
Best Trade [%]                      40.018423
Worst Trade [%]                    -14.327409
Avg. Trade [%]                    

In [86]:
bt.plot()


C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p5262', ...)

# Boillinger Bands

In [87]:
import numpy as np

In [88]:

class BollingerBands(Strategy):
    period = 20  # Standard period for Bollinger Bands
    std_dev = 2  # Number of standard deviations for the bands
    stop_loss_pct = 0.05  # Stop loss at 5%

    def init(self):
        # Compute moving average (SMA)
        self.ma = self.I(self.moving_average, self.data.Close, self.period)

        # Compute rolling standard deviation (compatible with Backtesting.py)
        self.std = self.I(self.rolling_std, self.data.Close, self.period)

        # Compute upper and lower Bollinger Bands
        self.upper_band = self.ma + self.std * self.std_dev
        self.lower_band = self.ma - self.std * self.std_dev

    def moving_average(self, data, period):
        """Simple moving average (SMA)"""
        return pd.Series(data).ewm(span=period, adjust=False).mean()


    def rolling_std(self, data, period):
        """Rolling standard deviation"""
        return pd.Series(data).rolling(window=period).std().dropna()

    def next(self):
        # If the price crosses below the lower band, we buy
        if crossover(self.lower_band, self.data.Close):
            if self.position.is_short:
                self.position.close()  # Close any short positions before buying
            self.buy()

        # If the price crosses above the upper band, we sell
        elif crossover(self.data.Close, self.upper_band):
            if self.position.is_long:
                self.position.close()  # Close any long positions before selling
            self.sell()

        # Implement stop loss
        if self.position:
            if self.position.is_long and self.data.Close[-1] < self.position.entry_price * (1 - self.stop_loss_pct):
                self.position.close()
            elif self.position.is_short and self.data.Close[-1] > self.position.entry_price * (1 + self.stop_loss_pct):
                self.position.close()

In [89]:
# Executando o backtest
bt = Backtest(tsla_data, BollingerBands, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

ValueError: Indicators must return (optionally a tuple of) numpy.arrays of same length as `data` (data shape: (411,); indicator "rolling_s…(C,20)"shape: (392,), returned value: [21.12497711 22.66282794 24.53174743 25.63072858 26.66551554 27.66614565
 28.36265275 29.2134584  28.75442577 27.56745535 27.46460469 26.86320578
 24.63736838 22.63120485 20.60147788 18.04141036 14.6056504  12.39042391
 12.00731626  9.88854048  7.93552503  6.99857594  6.43090637  6.13982018
  6.69798192  7.86871507  9.91108163 11.15191797 12.22327533 12.53804484
 12.48983605 11.67591656 11.73441114 11.12565488 11.1331945  10.83735187
 10.44823588 10.28742075  9.30768654  8.31426679  7.63499576  7.8274681
  8.78566337  8.82604632  8.90547142  8.8250677   8.14673324  7.42817529
  6.64276437  6.80255044  6.53164188  6.4960391   6.13679868  6.07977435
  6.15872155  8.47351101  9.79697372 11.09657241 12.22439167 13.81863384
 14.30241873 14.24071519 12.95027684 12.63805294 12.24335302 12.12880897
 11.71334629 11.27318